# 🤖 Customer Support Bot with Document Training & Feedback Loop for Serri AI

This notebook demonstrates the step-by-step development of an **AI-powered Customer Support Bot**.  
The bot is trained on a sample FAQ document, uses NLP models to answer queries, and improves responses through a feedback loop.  
Finally, we deploy the bot using **Gradio** for interactive use.  

### Workflow Steps:
1. **Environment Setup** – Install required libraries.  
2. **Imports & Logging** – Load dependencies and configure logging.  
3. **Bot Class Definition** – Implement `SupportBotAgent` with document ingestion, retrieval, QA, and feedback handling.  
4. **Initialize the Bot** – Load the FAQ and prepare embeddings.  
5. **Testing Queries** – Run sample queries with feedback loop.  
6. **Deploy Gradio Interface** – Build an interactive UI for deployment.  


## 📦 Install Required Packages
We start by installing the required libraries for the project, including **ipywidgets** for notebook interactivity.

In [4]:
pip install ipywidgets 

Note: you may need to restart the kernel to use updated packages.


Install **Sentence Transformers** for semantic embeddings and document retrieval.

In [1]:
pip install sentence_transformers

  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   --- ------------------------------------ 0.5/7.0 MB 1.4 MB/s eta 0:00:05
   ---- ----------------------------------- 0.8/7.0 MB 1.5 MB/s eta 0:00:05
   ------ --------------------------------- 1.0/7.0 MB 1.5 MB/s eta 0:00:04
   ------- -------------------------------- 1.3/7.0 MB 1.5 MB/s eta 0:00:04
   --------- ------------------------------ 1.6/7.0 MB 1.3 MB/s eta 0:00:05
   ---------- ----------------------------- 1.8/7.0 MB 1.3 MB/s eta 0:00:04
   --------------- ------------------------ 2.6/7.0 MB 1.6 MB/s eta 0:00:03
   ------------------ --------------------- 3.1/7.0 MB

Install **Gradio** for building a web-based user interface to interact with the bot.

In [3]:
pip install gradio

  Using cached gradio-5.47.2-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.118.0-py3-none-any.whl.metadata (28 kB)
  Using cached ffmpy-0.6.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.13.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.13.2-py3-none-win_amd64.whl.metadata (26 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.48.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tomlkit-0.13.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached typer-0.19.2-py3-none-any.whl.metadata (16 kB)


## 🛠 Import Libraries & Configure Environment
Here we import the required libraries:
- `transformers` for question-answering,
- `sentence-transformers` for semantic search,
- `torch` for backend,
- `gradio` for UI,  
and configure logging with timestamps.

In [4]:
import logging
import random
import torch
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import gradio as gr
from datetime import datetime

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

PyTorch Version: 2.8.0+cpu
CUDA Available: False


## 📑 Setup Logging
We configure logging to save all important actions, queries, and feedback into `support_bot_log.txt`.

In [5]:
# Setup logging
logging.basicConfig(
    filename='support_bot_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## 🤖 Define the SupportBotAgent Class
This class implements the entire workflow:
- **Document loading** (TXT/FAQ file)  
- **Semantic search** using embeddings  
- **Answer generation** using QA model  
- **Feedback simulation** and iterative improvement (max 2 iterations)  
- **Logging** of all decisions and actions  


In [6]:
class SupportBotAgent:
    """
    Intelligent Customer Support Bot with document training and feedback loop
    """
    
    def __init__(self, document_path):
        """Initialize the bot with document and models"""
        print("🤖 Initializing Support Bot...")
        
        # Load NLP models
        print("📥 Loading question-answering model...")
        self.qa_model = pipeline(
            "question-answering",
            model="distilbert-base-uncased-distilled-squad"
        )
        
        print("📥 Loading sentence embedding model...")
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Load and process document
        print(f"📄 Loading document: {document_path}")
        self.document_text = self.load_document(document_path)
        
        # Split into sections
        self.sections = [s.strip() for s in self.document_text.split('\n\n') if s.strip()]
        print(f"✂️  Document split into {len(self.sections)} sections")
        
        # Create embeddings
        print("🧮 Creating embeddings...")
        self.section_embeddings = self.embedder.encode(
            self.sections,
            convert_to_tensor=True
        )
        
        logging.info(f"Loaded document: {document_path} with {len(self.sections)} sections")
        print("✅ Bot initialized successfully!\n")
    
    def load_document(self, path):
        """Load document from file"""
        try:
            with open(path, 'r', encoding='utf-8') as file:
                text = file.read()
            return text
        except FileNotFoundError:
            logging.error(f"Document not found: {path}")
            return ""
        except Exception as e:
            logging.error(f"Error loading document: {str(e)}")
            return ""
    
    def find_relevant_section(self, query):
        """Find most relevant section using semantic search"""
        if not self.sections:
            return None
        
        query_embedding = self.embedder.encode(query, convert_to_tensor=True)
        similarities = util.cos_sim(query_embedding, self.section_embeddings)[0]
        best_idx = similarities.argmax().item()
        best_score = similarities[best_idx].item()
        
        logging.info(f"Query: '{query}' | Best match score: {best_score:.3f}")
        
        # Threshold for relevance
        if best_score < 0.3:
            logging.warning(f"Low relevance score for query: {query}")
            return None
        
        return self.sections[best_idx]
    
    def answer_query(self, query, iteration=0):
        """Generate answer for query"""
        context = self.find_relevant_section(query)
        
        if not context:
            response = "I don't have enough information to answer that question. Please contact our support team at support@example.com or call 1-800-555-1234."
            logging.info(f"No relevant context found for: {query}")
            return response
        
        try:
            result = self.qa_model(question=query, context=context)
            answer = result["answer"]
            confidence = result["score"]
            
            logging.info(f"Generated answer (confidence: {confidence:.3f}): {answer}")
            return answer
        except Exception as e:
            logging.error(f"Error generating answer: {str(e)}")
            return "I encountered an error processing your question. Please try rephrasing it."
    
    def get_feedback(self, response):
        """Simulate user feedback"""
        # Simulate realistic feedback distribution
        feedback = random.choices(
            ["not helpful", "too vague", "good"],
            weights=[0.2, 0.3, 0.5]  # 50% good, 30% vague, 20% not helpful
        )[0]
        
        logging.info(f"Feedback received: {feedback}")
        return feedback
    
    def adjust_response(self, query, response, feedback):
        """Adjust response based on feedback"""
        logging.info(f"Adjusting response based on feedback: {feedback}")
        
        if feedback == "too vague":
            # Add more context
            context = self.find_relevant_section(query)
            if context:
                adjusted = f"{response}\n\nAdditional context: {context[:200]}..."
                logging.info("Added context to response")
                return adjusted
        
        elif feedback == "not helpful":
            # Try rephrasing the query
            rephrased_query = f"{query} Please provide more details."
            adjusted = self.answer_query(rephrased_query, iteration=1)
            logging.info(f"Rephrased query: {rephrased_query}")
            return adjusted
        
        return response
    
    def process_query_with_feedback(self, query):
        """Process query with feedback loop (max 2 iterations)"""
        logging.info(f"\n{'='*60}")
        logging.info(f"NEW QUERY: {query}")
        logging.info(f"{'='*60}")
        
        # Initial response
        response = self.answer_query(query)
        history = [f"**Initial Response:** {response}"]
        
        # Feedback loop (max 2 iterations)
        for iteration in range(2):
            feedback = self.get_feedback(response)
            
            if feedback == "good":
                history.append(f"\n**Feedback {iteration + 1}:** ✅ Good - Response accepted")
                break
            
            history.append(f"\n**Feedback {iteration + 1}:** ⚠️ {feedback.title()}")
            response = self.adjust_response(query, response, feedback)
            history.append(f"**Adjusted Response:** {response}")
        
        return "\n".join(history)
    
    def chat(self, query):
        """Simple chat interface without feedback simulation"""
        if not query.strip():
            return "Please enter a question."
        
        logging.info(f"Chat query: {query}")
        response = self.answer_query(query)
        return response

print("✅ SupportBotAgent class defined!")

✅ SupportBotAgent class defined!


## 🚀 Initialize the Bot
We create an instance of `SupportBotAgent` and load the sample FAQ document (`faq.txt`).

In [11]:
# Initialize the bot
bot = SupportBotAgent("faq.txt")

🤖 Initializing Support Bot...
📥 Loading question-answering model...


Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu


📥 Loading sentence embedding model...
📄 Loading document: faq.txt
✂️  Document split into 10 sections
🧮 Creating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Bot initialized successfully!



## 🧪 Test the Bot with Sample Queries
We run a few example queries (e.g., password reset, refund policy, out-of-scope queries) and observe how the bot handles feedback and adjusts its responses.

In [12]:
# Test queries
test_queries = [
    "How do I reset my password?",
    "What's the refund policy?",
    "How do I contact support?",
    "What are the shipping options?",
    "How do I fly to the moon?"  # Out-of-scope
]

print("🧪 Testing bot with sample queries...\n")
for query in test_queries:
    print(f"\n{'='*70}")
    print(f"Query: {query}")
    print(f"{'='*70}")
    result = bot.process_query_with_feedback(query)
    print(result)
    print()

🧪 Testing bot with sample queries...


Query: How do I reset my password?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**Initial Response:** go to the login page

**Feedback 1:** ⚠️ Too Vague
**Adjusted Response:** go to the login page

Additional context: Resetting Your Password
To reset your password, go to the login page and click "Forgot Password." 
Enter your email address and follow the link sent to your inbox. 
If you don't receive the email with...

**Feedback 2:** ✅ Good - Response accepted


Query: What's the refund policy?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

**Initial Response:** free for orders over $50

**Feedback 1:** ✅ Good - Response accepted


Query: How do I fly to the moon?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**Initial Response:** I don't have enough information to answer that question. Please contact our support team at support@example.com or call 1-800-555-1234.

**Feedback 1:** ⚠️ Not Helpful
**Adjusted Response:** I don't have enough information to answer that question. Please contact our support team at support@example.com or call 1-800-555-1234.

**Feedback 2:** ⚠️ Too Vague
**Adjusted Response:** I don't have enough information to answer that question. Please contact our support team at support@example.com or call 1-800-555-1234.



## 🌐 Create Gradio Interface for Deployment
Here we build a Gradio interface with three tabs:
- **Chat**: Ask questions and get answers directly.  
- **Feedback Simulation**: See how responses improve with simulated feedback.  
- **About**: Information about the bot and its features.  

Finally, we launch the Gradio demo and generate a shareable public link.

In [13]:
# Create Gradio Interface
def create_gradio_interface():
    """Create Gradio UI for the bot"""
    
    # Custom CSS
    custom_css = """
    .gradio-container {
        font-family: 'Arial', sans-serif;
    }
    .header {
        text-align: center;
        padding: 20px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 10px;
        margin-bottom: 20px;
    }
    """
    
    with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
        
        gr.Markdown(
            """
            <div class="header">
                <h1>🤖 AI Customer Support Bot</h1>
                <p>Ask me anything about our services, policies, and support!</p>
            </div>
            """
        )
        
        with gr.Tab("💬 Chat"):
            gr.Markdown("### Ask your question and get instant answers!")
            
            with gr.Row():
                with gr.Column(scale=2):
                    query_input = gr.Textbox(
                        label="Your Question",
                        placeholder="e.g., How do I reset my password?",
                        lines=2
                    )
                    
                    with gr.Row():
                        submit_btn = gr.Button("Ask Question", variant="primary", size="lg")
                        clear_btn = gr.Button("Clear", size="lg")
                    
                    # Example queries
                    gr.Examples(
                        examples=[
                            "How do I reset my password?",
                            "What is your refund policy?",
                            "How can I contact support?",
                            "What are the shipping options?",
                            "How do I track my order?",
                            "What payment methods do you accept?"
                        ],
                        inputs=query_input,
                        label="Try these example questions:"
                    )
                
                with gr.Column(scale=3):
                    response_output = gr.Textbox(
                        label="Bot Response",
                        lines=10,
                        interactive=False
                    )
        
        with gr.Tab("🔄 Feedback Simulation"):
            gr.Markdown("### See how the bot improves responses with simulated feedback!")
            
            query_feedback = gr.Textbox(
                label="Your Question",
                placeholder="e.g., What's the refund policy?",
                lines=2
            )
            
            feedback_btn = gr.Button("Process with Feedback Loop", variant="primary", size="lg")
            
            feedback_output = gr.Textbox(
                label="Response with Feedback Iterations",
                lines=15,
                interactive=False
            )
        
        with gr.Tab("📊 About"):
            gr.Markdown(
                """
                ## About This Bot
                
                This intelligent customer support bot uses:
                
                - **📄 Document Training**: Trained on company FAQ documents
                - **🧠 NLP Models**: 
                  - `distilbert-base-uncased-distilled-squad` for question answering
                  - `all-MiniLM-L6-v2` for semantic search
                - **🔍 Semantic Search**: Finds relevant information using embeddings
                - **🔄 Feedback Loop**: Improves responses based on simulated feedback
                - **📝 Logging**: Tracks all decisions and actions
                
                ### Features
                - ✅ Accurate answers from trained documents
                - ✅ Handles out-of-scope queries gracefully
                - ✅ Iterative improvement with feedback
                - ✅ Transparent decision logging
                
                ### Technology Stack
                - Python 3.8+
                - Transformers (Hugging Face)
                - Sentence Transformers
                - Gradio
                
                ---
                
                **Note**: This is a demo bot. For real support, contact support@example.com
                """
            )
        
        # Event handlers
        submit_btn.click(
            fn=bot.chat,
            inputs=query_input,
            outputs=response_output
        )
        
        feedback_btn.click(
            fn=bot.process_query_with_feedback,
            inputs=query_feedback,
            outputs=feedback_output
        )
        
        clear_btn.click(
            lambda: ("", ""),
            outputs=[query_input, response_output]
        )
    
    return demo

# Create and launch interface
demo = create_gradio_interface()
demo.launch(share=True)  # share=True creates public link

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://20cb0f43be758f7ebf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
